In [136]:
#23/5/24 For extracting latent space from a autoencoder model
#Outputs the model and a text file

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
targetdata = pd.read_csv('/Users/james/Documents/Honours/Data/Targetdata/DrugTargets.csv')
#autoencoder = keras.models.load_model('/Users/james/Documents/Honours/Python/Target_Selection/Drug_Targets/autoencode/bestmodel/models copy/AE-7.model')
autoencoder = keras.models.load_model('/Users/james/Documents/Honours/Python/Target_Selection/Drug_Targets/autoencode/models/AE-7.keras')

from sklearn.model_selection import train_test_split

trainset, testset = train_test_split(targetdata, test_size=0.2, random_state=81)
#Converts Toxicity Values to a list
ytrain = trainset['Toxicity_Values']
ytest = testset['Toxicity_Values']
#Converts encoded drug target values to an array
xtrain = trainset.iloc[:, 2:].values
xtest = testset.iloc[:, 2:].values

In [146]:
#Test model metrics
colcount = len(xtest[0])
ae_out = autoencoder.predict([ xtest.reshape(-1, colcount, 1) ])
predictions = ae_out.reshape(2649,colcount)

correctcount = 0
fpcount = 0
tpcount = 0
tncount = 0
fncount = 0
testpos = 0
testneg = 0

iterations = 0
for samples in predictions:
    loops = 0
    for value in predictions[iterations]:
        testscore = xtest[iterations][loops]
        if value > 0.5:
            if testscore != 0:
                testpos = testpos + 1
                correctcount = correctcount + 1
                tpcount = tpcount + 1
            else:
                fpcount = fpcount + 1
                testneg = testneg + 1
        else:
            if testscore != 0:
                testpos = testpos + 1
                fncount = fncount + 1
            else:
                testneg = testneg + 1
                correctcount = correctcount + 1
                tncount = tncount + 1
                
        loops = loops + 1

print('model = 8.model')

print('positives in data', testpos)
print('negatives in data', testneg)

print('fn count =', fncount)
print('tn count =', tncount)

print('tp count =', tpcount)
print('fp count =', fpcount)

netfn = fncount / (fncount + tncount)
nettn = tncount / (fncount + tncount)
netacc = correctcount / (fpcount + fncount + tpcount + tncount)
posacc = tpcount / testpos
negacc = tncount / testneg
netfp = fpcount / (fpcount + tpcount)
nettp = tpcount / (tpcount + fpcount)
recall = tpcount / (tpcount + fncount)
f1 = 2 * ((recall * nettp)/(recall + nettp))

print('fn rate =', netfn)
print('tn rate =', nettn)

print('net accuracy =', netacc)
print('positive accuracy =', posacc)
print('negative accuracy =', negacc)

print('fp rate =', netfp)
print('precision =', nettp)
print('recall =',recall)
print('f1 score =',f1)


83/83 [==============================] - 0s 375us/step
model = 8.model
positives in data 13245
negatives in data 871521
fn count = 2649
tn count = 868872
tp count = 10596
fp count = 2649
fn rate = 0.00303951367781155
tn rate = 0.9969604863221885
net accuracy = 0.9940119760479041
positive accuracy = 0.8
negative accuracy = 0.9969604863221885
fp rate = 0.2
precision = 0.8
recall = 0.8
f1 score = 0.8000000000000002


In [124]:
#Notabely 0.5 was used as the cutoff point for determining if a cell is 0 or 1
#As the metrics are very good, we will extract the latent space for prediction 
features = autoencoder.layers[3].output
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 334, 1)]          0         
                                                                 
 flatten (Flatten)           (None, 334)               0         
                                                                 
 dense (Dense)               (None, 160)               53600     
                                                                 
 dense_1 (Dense)             (None, 81)                13041     
                                                                 
 dense_2 (Dense)             (None, 42)                3444      
                                                                 
 dense_3 (Dense)             (None, 42)                1806      
                                                                 
 dense_4 (Dense)             (None, 81)                

In [125]:
ae_out.shape

(2649, 334, 1)

In [134]:
#Extracts data from latent space
from keras import backend as K
latent_space = K.function([autoencoder.layers[0].input],
                                  [autoencoder.layers[3].output])
output = latent_space([xtrain])[0]
latent = pd.DataFrame(output)

toxvals= []
for values in ytrain:
    toxvals.append(values)
temp = pd.DataFrame(newlist, columns=['Toxicity_Value'])

latentframe = pd.concat([temp, latent], axis=1)
latentframe.to_csv('/Users/james/Documents/Honours/Data/Targetdata/autoencdata/dtargetautoencdata.csv', index=False)

In [139]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

testout = latent_space([xtest])[0]
latentest = pd.DataFrame(testout)

svc = SVC().fit(xtest, ytest)
print('svc raw =',svc.score(xtest, ytest))
knn = KNeighborsClassifier().fit(xtest, bling)
print('knn raw =',knn.score(xtest, bling))

svc = SVC().fit(latentest, ytest)
print('svc autoenc =',svc.score(latentest, ytest))
knn = KNeighborsClassifier().fit(latentest.values, bling)
print('knn autoenc =',knn.score(latentest.values, bling))

#It seems that the latent space data actually performed worse than the raw dataset
#Although it was using default hyperparameters so potentially it may perform better when optimised

svc raw = 0.6949792374480936
knn raw = 0.6987542468856173


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


svc autoenc = 0.5892789731974329
knn autoenc = 0.6949792374480936


/opt/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
